In [1]:
import csv
import sys
import numpy as np
from sklearn.naive_bayes import GaussianNB
import urllib
import random
from sklearn.preprocessing import LabelEncoder  
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline, FeatureUnion
from numpy import genfromtxt
import sklearn
%matplotlib inline
from sklearn.metrics import mean_absolute_error
import json
import os
import scipy.optimize
from math import exp
from math import log
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import movie_reviews
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from collections import defaultdict
from sklearn import linear_model
from sklearn.linear_model import Ridge
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse
import string
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor#GBM algorithm
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from sklearn.preprocessing import MultiLabelBinarizer
import random
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfTransformer


C:\Anaconda\envs\python2\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
#Reading question data
op = []
print sys.maxsize
#csv.field_size_limit(sys.maxsize)
with open('../Questions.csv', 'rb') as csvfile:
    spamreader = csv.reader(csvfile)
    for row in spamreader:
        op.append(row)

9223372036854775807


In [9]:
myFile = open('../Questions_first_100.csv', 'w')  
with myFile:  
   writer = csv.writer(myFile)
   writer.writerows(op[0:100])

In [11]:
#Reading tags data
tags = []
with open('../Tags.csv', 'rb') as csvfile:
    spamreader = csv.reader(csvfile)
    for row in spamreader:
        tags.append(row)

In [12]:
myFile = open('../Tags_first_100.csv', 'w')  
with myFile:  
   writer = csv.writer(myFile)
   writer.writerows(tags[0:100])

In [13]:
tagMod = tags[1:]
tagList = defaultdict(list)
countList = defaultdict(int)
for tag in tagMod:
    tagList[tag[0]].append(tag[1])
    countList[tag[1]]+=1

counts = [(countList[tag],tag) for tag in countList]
counts.sort()
counts.reverse()
finalTags = [x[1] for x in counts[0:500]]

In [14]:
qlist = []
finaldatax = []
quest_tag_dict = {}
for data in op:
    tags = tagList[data[0]]
    if len(list(set(tags) & set(finalTags))) == len(list(set(tags))) and len(list(set(tags))) > 0 :
        quest_tag_dict[data[0]] = list(set(tags) & set(finalTags))
        finaldatax.append(data)
random.shuffle(finaldatax)

In [15]:
punctuation = set(string.punctuation)

In [16]:
finaldata = finaldatax[:300000]

In [17]:
def getY(data):
    Y = np.zeros((len(data),len(finalTags)))
    i = 0
    for d in data:
        for tag in finalTags:
            if tag in tagList[d[0]]:
                Y[i][tag_to_ix[tag]] = 1
        i+=1
    return Y

In [18]:
tag_to_ix = { ch:i for i,ch in enumerate(finalTags) }
ix_to_tag = { i:ch for i,ch in enumerate(finalTags) }
tagID = dict(zip(finalTags,range(len(finalTags))))
def feature(datum):
    feat = [0]*20
    words = datum[5].split()
    body = datum[6].split()
    words.extend(body)
    for w in words:
        if w in finalTags:
            feat[tagID[w]]+=1
    return feat

def getFeatures(data):
    X = [feature(d) for d in data]
    return X

In [19]:
def getMse(prediction_actual,prediction_test):
    mse=[]
    for tag in finalTags:
        me=mean_absolute_error(prediction_test[:,tag_to_ix[tag]],prediction_actual[:,tag_to_ix[tag]])
        mse.append(me)
    return sum(mse)/(len(mse))   

def getPrecisionRecall(Y, k):
    prec = []
    rec = []
    f1 = []
    for x in Y:
        temp = x[:k]
        p = sum(i[1] for i in temp) * 1.0 / k
        r = sum(i[1] for i in temp) * 1.0 / sum(i[1] for i in x)
        if p+r <= 0:
            f = 0
        else:
            f = 2*p*r*1.0/(p+r)
        prec.append(p)
        rec.append(r)
        f1.append(f)
    return prec, rec, f1

In [20]:
data_train = finaldata[:int(0.75*len(finaldata))]
data_test = finaldata[int(0.75*len(finaldata)):]

In [21]:
X_train = [(x[5] + " " + x[6]).lower() for x in data_train]
X_test = [(x[5]+ " " + x[6]).lower() for x in data_test]

In [ ]:
X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = getY(data_train)
Y_test = getY(data_test)

print "Shapes"
print X_train.shape
print Y_train.shape
print X_test.shape
print Y_test.shape

Shapes
(225000L,)
(225000L, 500L)
(75000L,)
(75000L, 500L)


In [ ]:
classifier = Pipeline([
    ('vectorizer', CountVectorizer(stop_words = 'english',token_pattern = '\S+')),
    ('tfidf', TfidfTransformer()),
    ('clf', OneVsRestClassifier(SGDClassifier()))])
classifier.fit(X_train, Y_train)

C:\Anaconda\envs\python2\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [ ]:
predicted = classifier.predict(X_test)
scores = classifier.decision_function(X_test)

In [ ]:
final_op1 = []
i = 0
for i in range(0,predicted.shape[0]):
    temp = (zip(scores[i],Y_test[i],predicted[i]))
    temp.sort()
    temp.reverse()
    final_op1.append(temp)

In [ ]:
k = 3
prec, rec, f1 = getPrecisionRecall(final_op1, k)

In [ ]:
prec_mean = sum(prec)/float(len(prec))
rec_mean = sum(rec)/float(len(rec))
f1_mean = sum(f1)/float(len(f1))

In [ ]:
print "F1 (@k) :: ", f1_mean
print "Recall (@k) :: ",rec_mean
print "Precision (@k) :: ",prec_mean
print "MSE :: ",getMse(Y_test,predicted)

In [ ]:
X_axis = []
# F1 (@k) ::  0.412167904762
# Recall (@k) ::  0.727331333333
# Precision (@k) ::  0.299506666667
# MSE ::  0.029076
prec_axis = []
rec_axis = []
f1_axis = []
for i in range(1,20):
    X_axis.append(i)
    prec, rec, f1 = getPrecisionRecall(final_op1, i)
    prec_axis.append(sum(prec)/float(len(prec)))
    rec_axis.append(sum(rec)/float(len(rec)))
    f1_axis.append(sum(f1)/float(len(f1)))



In [ ]:
fig = plt.figure()
ax = plt.axes()        
ax.yaxis.grid(linestyle='--') # horizontal lines
plt.xticks(range(1,21))
plt.plot(X_axis, prec_axis)
plt.plot(X_axis, rec_axis)
plt.plot(X_axis, f1_axis)
# plt.grid(linestyle='-')
plt.legend(['Precision','Recall','F1 Score'],loc='lower left')
fig.suptitle('Performance vs Number of predicted tags')
plt.xlabel('Number of predicted tags')
# plt.ylabel('Frequency')

In [ ]:
vectorizer = TfidfVectorizer(token_pattern='\S+',min_df=1)
vectorizer.fit_transform(["kfg ufg qeufg uiefg jq.ui 78 eufg euf","qkjfgqwufg qufg qu"])

In [ ]:
vectorizer.get_feature_names()